##### 台積電，鴻海 ，大立光 #####

In [1]:
import os
import numpy as np
import pandas as pd
import re,random
import jieba
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
stop_list=[]
with open('./stopword.txt','r',encoding='utf-8') as f:
    for line in f.readlines():
        stop_list.append(line.strip())

In [3]:
def preprocess(data,all_data,category):
    for idx, line in enumerate(data):
        line = re.sub(r'[^\w]','',line)
        line = re.sub(r'[A-Za-z0-9]','',line)
        line = re.sub(u'[\uFF01-\uFF5A]','',line)
        line = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))','',line)
        segment_list = jieba.lcut(line)
        segment_list = filter(lambda x: len(x)>1,segment_list)
        segment_list = filter(lambda x: x not in stop_list,segment_list)
        all_data.append( (' '.join(segment_list),category[idx]) )
    return all_data

In [4]:
def load_data(fileName):
    filePath = "./" + fileName + "/"
    pos = pd.read_csv(filePath + os.listdir(filePath)[2])
    neg = pd.read_csv(filePath + os.listdir(filePath)[1])
    pre_data = pd.concat([pos,neg],axis=0)
    data = []
    text = pd.DataFrame(columns=["text","label"])
    text["text"] = pre_data["title"].fillna("")+pre_data["content"].fillna("")
    text["label"] = pre_data["label"] 
    text= text.reset_index(drop=True)
    data =  preprocess(text["text"],data,text["label"])
    return data

In [5]:
fileName = '台積電'
data = load_data(fileName)
random.shuffle(data)
x,y = zip(*data)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=666)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xiaoy\AppData\Local\Temp\jieba.cache
Loading model cost 0.600 seconds.
Prefix dict has been built successfully.


In [6]:
vec = CountVectorizer(
    analyzer='word', 
    ngram_range=(2,6), 
    max_features=8000
)
vec.fit(x_train)

tvec = TfidfVectorizer(
    analyzer='word',
    ngram_range=(2,6), 
    max_features=8000 
)
tvec.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=8000,
                min_df=1, ngram_range=(2, 6), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [12]:
clf = tree.DecisionTreeClassifier() 
clf = clf.fit(tvec.transform(x_train),y_train)
result = clf.score(tvec.transform(x_test),y_test)
print(result)

0.6805225653206651


In [53]:
print(clf.predict(tvec.transform(['資本 美元 創歷史 目標 價元 資挺鴻海 目標 外資'])))

[1]


In [8]:
gbdt = GradientBoostingClassifier()
gbdt = gbdt.fit(tvec.transform(x_train),y_train)
result = gbdt.score(tvec.transform(x_test),y_test)
print(result)

0.6963578780680918
